# For converting REvIEW output to $q^2$ file input format

In [16]:
import os
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from matplotlib import rcParams

# RC params
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 14
rcParams["axes.edgecolor"] = 'black'
rcParams["legend.edgecolor"] = '0.8'
plt.rcParams.update({'errorbar.capsize': 2})
plt.rcParams['mathtext.fontset'] = 'dejavuserif'
plt.rcParams['text.latex.preamble']=r'\usepackage{amsmath}'

In [44]:
review_path = '/Users/maddie/Desktop/M22/equivelant widths/REvIEW updates'

line_list_path = '/Users/maddie/Documents/steph_daospec/line lists'
line_list_name = 'stephlinelist2.dat'

review_output_name = 'fit_outputs_checked3.csv'
q2_input_name = 'review_update_lines3.csv'

In [45]:
os.chdir(line_list_path)

ll = pd.read_table(line_list_name, sep="\s+", header=0, usecols = [0, 1, 2, 3], names = ['wavelength', 'species', 'ep', 'gf'])
ll.wavelength = np.round(ll.wavelength, 4)

ll['join_line'] = ll.wavelength * 10000
ll['join_line'] = ll['join_line'].astype(int)

In [46]:
# Read in my csv
os.chdir(review_path)
review_fits = pd.read_csv(review_output_name)


print('Array dimentions before cutting lines', review_fits.shape)

review_fits.line = np.round(review_fits.line, 4)

# Various cuts I want to make on the data
review_fits = review_fits[review_fits.flag != 4] # where the continuum was wrong
review_fits = review_fits[review_fits.flag != 5] # manually removed

review_fits = review_fits[(review_fits.ew > 5) & (review_fits.ew < 180)]


print('Array dimentions after cutting lines', review_fits.shape)

# Creating the line to merge on
review_fits['join_line'] = review_fits.line * 10000
review_fits['join_line'] = review_fits['join_line'].astype(int)

review_fits_ll = review_fits.merge(ll, on = 'join_line')

# Separating out all the stars into their own arrays
scombc   = review_fits_ll[(review_fits_ll.star_name == 'scomb2.M22_C.U.txt')       | \
                          (review_fits_ll.star_name == 'scomb2.M22_C.L.txt')]
scomb3   = review_fits_ll[(review_fits_ll.star_name == 'scomb2.M22_III-3.U.txt')   | \
                          (review_fits_ll.star_name == 'scomb2.M22_III-3.L.txt')]
scomb14  = review_fits_ll[(review_fits_ll.star_name == 'scomb2.M22_III-14.U.txt')  | \
                          (review_fits_ll.star_name == 'scomb2.M22_III-14.L.txt')]
scomb15  = review_fits_ll[(review_fits_ll.star_name == 'scomb2.M22_III-15.U.txt')  | \
                          (review_fits_ll.star_name == 'scomb2.M22_III-15.L.txt') ]
scomb52  = review_fits_ll[(review_fits_ll.star_name == 'scomb2.M22_III-52.U.txt')  | \
                          (review_fits_ll.star_name == 'scomb2.M22_III-52.L.txt')]
scomb102 = review_fits_ll[(review_fits_ll.star_name == 'scomb2.M22_IV-102.U.txt')  | \
                          (review_fits_ll.star_name == 'scomb2.M22_IV-102.L.txt')]
ref      = review_fits_ll[review_fits_ll.star_name == 'reference_6752.txt']	

Array dimentions before cutting lines (3766, 9)
Array dimentions after cutting lines (2519, 9)


In [47]:
print(review_fits.shape)
print(review_fits_ll.shape)

(2519, 10)
(2519, 14)


In [48]:
review_fits

,line,star_name,flux_ratio,centre,sigma,ew,chi2,d,flag,join_line
1,4786.530,scomb2.M22_III-3.L.txt,-0.431019,4786.533517,0.066931,72.312851,0.073174,0.921183,0,47865300
2,4786.530,scomb2.M22_III-14.L.txt,-0.410584,4786.533492,0.065489,67.400098,0.020286,0.977472,0,47865300
6,4786.530,reference_6752.txt,-0.514109,4786.549144,0.059705,76.940840,0.023017,0.996581,0,47865300
8,4786.807,scomb2.M22_III-3.L.txt,-0.505820,4786.795707,0.099057,125.594829,0.041972,0.947735,0,47868070
9,4786.807,scomb2.M22_III-14.L.txt,-0.554746,4786.801821,0.080803,112.359756,0.022120,0.986754,0,47868070
...,...,...,...,...,...,...,...,...,...,...
3753,6774.268,scomb2.M22_III-3.U.txt,-0.115665,6774.207395,0.123671,35.855677,0.000293,0.999526,2,67742680
3754,6774.268,scomb2.M22_III-14.U.txt,-0.079615,6774.217945,0.106586,21.270956,0.000767,0.991181,0,67742680
3756,6774.268,scomb2.M22_III-52.U.txt,-0.139161,6774.219158,0.100541,35.071332,0.000637,0.997910,2,67742680
3757,6774.268,scomb2.M22_IV-102.U.txt,-0.062744,6774.197564,0.122450,19.258382,0.000270,1.003008,2,67742680


In [49]:
q2_line = ll.merge(scombc[['join_line', 'ew']], how = 'left',on='join_line')
q2_line.rename({'ew': 'C'}, axis=1, inplace=True)

q2_line = q2_line.merge(scomb3[['join_line', 'ew']], how = 'left',on='join_line')
q2_line.rename({'ew': 'III_3'}, axis=1, inplace=True)

q2_line = q2_line.merge(scomb14[['join_line', 'ew']], how = 'left',on='join_line')
q2_line.rename({'ew': 'III_14'}, axis=1, inplace=True)

q2_line = q2_line.merge(scomb15[['join_line', 'ew']], how = 'left',on='join_line')
q2_line.rename({'ew': 'III_15'}, axis=1, inplace=True)

q2_line = q2_line.merge(scomb52[['join_line', 'ew']], how = 'left',on='join_line')
q2_line.rename({'ew': 'III_52'}, axis=1, inplace=True)

q2_line = q2_line.merge(scomb102[['join_line', 'ew']], how = 'left',on='join_line')
q2_line.rename({'ew': 'IV_102'}, axis=1, inplace=True)

q2_line = q2_line.merge(ref[['join_line', 'ew']], how = 'left',on='join_line')
q2_line.rename({'ew': 'REF'}, axis=1, inplace=True)

In [50]:
q2_line = q2_line.round({'C': 3, 'III_3': 3, 'III_14': 3, 'III_15': 3, 'III_52': 3, 'IV_102': 3, 'REF': 3})

In [51]:
os.chdir(review_path)
q2_line.to_csv('M22_lines_REVIEW3.csv', index=False)

In [52]:
q2_line

,wavelength,species,ep,gf,join_line,C,III_3,III_14,III_15,III_52,IV_102,REF
0,6363.78,8.0,0.02,-10.30,63637800,28.937,35.280,24.157,NaN,27.695,22.026,12.427
1,5682.63,11.0,2.10,-0.71,56826300,103.326,58.536,37.422,71.621,59.880,41.683,57.332
2,5688.20,11.0,2.10,-0.41,56882000,131.177,84.318,63.215,102.712,89.734,59.336,83.274
3,5889.95,11.0,0.00,0.11,58899500,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5895.92,11.0,0.00,-0.19,58959200,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
542,5255.51,60.1,0.20,-0.67,52555100,97.903,96.218,67.776,47.242,76.016,64.996,44.409
543,5293.16,60.1,0.82,0.10,52931600,NaN,NaN,51.511,48.232,59.981,43.273,40.980
544,5319.81,60.1,0.55,-0.14,53198100,77.665,89.376,65.914,56.931,73.879,58.045,53.188
545,4844.21,62.1,0.28,-0.89,48442100,47.389,43.615,32.946,31.255,35.714,25.765,22.663


# Removing $q^2$ outliers

In [54]:
q2_output_dir = '/Users/maddie/Desktop/M22/q2 output/q2_FeI_output2'

In [80]:
os.chdir(q2_output_dir)

scomb3_FeI = pd.read_csv('3-Table 1.csv', names=['wavelength', 'absolute', 'abs_res', 'dif', 'dif_abs'], header =0)
scomb14_FeI = pd.read_csv('14-Table 1.csv', names=['wavelength', 'absolute', 'abs_res', 'dif', 'dif_abs'], header =0)
scomb15_FeI = pd.read_csv('15-Table 1.csv', names=['wavelength', 'absolute', 'abs_res', 'dif', 'dif_abs'], header =0)
scomb52_FeI = pd.read_csv('52-Table 1.csv', names=['wavelength', 'absolute', 'abs_res', 'dif', 'dif_abs'], header =0)
scomb102_FeI = pd.read_csv('102-Table 1.csv', names=['wavelength', 'absolute', 'abs_res', 'dif', 'dif_abs'], header =0)
scombc_FeI = pd.read_csv('C-Table 1.csv', names=['wavelength', 'absolute', 'abs_res', 'dif', 'dif_abs'], header =0)

In [223]:
def remove_sigma_lines(fe_arr, total_arr):

    tmp_arr = []
    sigma = np.std(fe_arr.dif)
    mean  = np.mean(fe_arr.dif)

    lines_to_cut = (fe_arr[(fe_arr.dif> mean + (2* sigma)) | (fe_arr.dif< mean - (2* sigma))].wavelength)
    append = True
    #print(lines_to_cut)
    for i in range(total_arr.shape[0]):
        for j in lines_to_cut:
            if abs(total_arr.line.iloc[i] - j) < 0.1:
                append = False

        if append:
            tmp_arr.append(total_arr.iloc[i])
        append = True
    
    if total_arr.shape[0] - len(lines_to_cut) != len(tmp_arr):
        print('wrong shape')

    return pd.DataFrame(tmp_arr)

In [224]:
scombc_new = remove_sigma_lines(scombc_FeI, scombc)
scomb3_new = remove_sigma_lines(scomb3_FeI, scomb3)
scomb14_new = remove_sigma_lines(scomb14_FeI, scomb14)
scomb15_new = remove_sigma_lines(scomb15_FeI, scomb15)
scomb52_new = remove_sigma_lines(scomb52_FeI, scomb52)
scomb102_new = remove_sigma_lines(scomb102_FeI, scomb102)

In [225]:
q2_line_new = ll.merge(scombc_new[['join_line', 'ew']], how = 'left',on='join_line')
q2_line_new.rename({'ew': 'C'}, axis=1, inplace=True)

q2_line_new = q2_line_new.merge(scomb3_new[['join_line', 'ew']], how = 'left',on='join_line')
q2_line_new.rename({'ew': 'III_3'}, axis=1, inplace=True)

q2_line_new = q2_line_new.merge(scomb14_new[['join_line', 'ew']], how = 'left',on='join_line')
q2_line_new.rename({'ew': 'III_14'}, axis=1, inplace=True)

q2_line_new = q2_line_new.merge(scomb15_new[['join_line', 'ew']], how = 'left',on='join_line')
q2_line_new.rename({'ew': 'III_15'}, axis=1, inplace=True)

q2_line_new = q2_line_new.merge(scomb52_new[['join_line', 'ew']], how = 'left',on='join_line')
q2_line_new.rename({'ew': 'III_52'}, axis=1, inplace=True)

q2_line_new = q2_line_new.merge(scomb102_new[['join_line', 'ew']], how = 'left',on='join_line')
q2_line_new.rename({'ew': 'IV_102'}, axis=1, inplace=True)

q2_line_new = q2_line_new.merge(ref[['join_line', 'ew']], how = 'left',on='join_line')
q2_line_new.rename({'ew': 'REF'}, axis=1, inplace=True)

In [227]:
os.chdir(review_path)
q2_line_new.to_csv('M22_lines_REVIEW4.csv', index=False)